# Horvath2013

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.Horvath2013)

class Horvath2013(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        """
        Applies an anti-logarithmic linear transformation to a PyTorch tensor.
        """
        adult_age = 20

        # Create a mask for negative and non-negative values
        mask_negative = x < 0
        mask_non_negative = ~mask_negative

        # Initialize the result tensor
        age_tensor = torch.empty_like(x)

        # Exponential transformation for negative values
        age_tensor[mask_negative] = (1 + adult_age) * torch.exp(x[mask_negative]) - 1

        # Linear transformation for non-negative values
        age_tensor[mask_non_negative] = (1 + adult_age) * x[
            mask_non_negative
        ] + adult_age

        return age_tensor



In [3]:
model = pya.models.Horvath2013()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'horvath2013'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2013
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Horvath, Steve. \"DNA methylation age of human tissues and cell types.\" Genome biology 14.10 (2013): 1-20."
model.metadata["doi"] = "https://doi.org/10.1186/gb-2013-14-10-r115"
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

#### Download directly with curl

In [5]:
supplementary_url = "https://static-content.springer.com/esm/art%3A10.1186%2Fgb-2013-14-10-r115/MediaObjects/13059_2013_3156_MOESM3_ESM.csv"
supplementary_file_name = "coefficients.csv"
os.system(f"curl -o {supplementary_file_name} {supplementary_url}")

0

In [6]:
supplementary_url = "https://static-content.springer.com/esm/art%3A10.1186%2Fgb-2013-14-10-r115/MediaObjects/13059_2013_3156_MOESM22_ESM.csv"
supplementary_file_name = "reference_feature_values.csv"
os.system(f"curl -o {supplementary_file_name} {supplementary_url}")

0

## Load features

#### From CSV file

In [7]:
df = pd.read_csv('coefficients.csv', skiprows=2)
df['feature'] = df['CpGmarker']
df['coefficient'] = df['CoefficientTraining']
model.features = df['feature'][1:].tolist()

## Load weights into base model

In [8]:
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][0]])

#### Linear model

In [9]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

#### From CSV file

In [10]:
reference_feature_values_df = pd.read_csv('reference_feature_values.csv', index_col=0)
reference_feature_values_df = reference_feature_values_df.loc[model.features]
model.reference_values = reference_feature_values_df['goldstandard2'].tolist()

## Load preprocess and postprocess objects

In [11]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [12]:
model.postprocess_name = 'anti_log_linear'
model.postprocess_dependencies = None

## Check all clock parameters

In [13]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Horvath, Steve. "DNA methylation age of human tissues and cell '
             'types." Genome biology 14.10 (2013): 1-20.',
 'clock_name': 'horvath2013',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1186/gb-2013-14-10-r115',
 'notes': None,
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2013}
reference_values: [0.790221397, 0.89001929, 0.059106387, 0.23651937, 0.073668777, 0.563295909, 0.864999404, 0.027047887, 0.660721193, 0.033420176, 0.047913033, 0.517283973, 0.050756537, 0.072267723, 0.014877693, 0.876157036, 0.187082703, 0.170303406, 0.019217238, 0.560726569, 0.845964086, 0.447995921, 0.055406903, 0.059821557, 0.533869814, 0.065190933, 0.896227421, 0.090932158, 0.032431793, 0.480007151]... [Total elements: 353]
preprocess_name: None
preprocess_dependencies: None
p

## Basic test

In [14]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[ -0.9718],
        [  1.8615],
        [ -0.9980],
        [ 67.1194],
        [253.6647],
        [ 61.3025],
        [ -0.9928],
        [ -0.6078],
        [ -0.9995],
        [ -0.7832]], dtype=torch.float64, grad_fn=<IndexPutBackward0>)

## Save torch model

In [15]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [16]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: coefficients.csv
Deleted file: reference_feature_values.csv
